# Experiment on the robustness of ensemble VIT(Basic)+Resnet(Projection)




## 1.Setup

In [ ]:
# basic settings
!pip install timm
!pip install adversarial-robustness-toolbox
!pip install timm foolbox

     |████████████████████████████████| 296kB 7.1MB/s 
     |████████████████████████████████| 972kB 7.2MB/s 
     |████████████████████████████████| 21.6MB 1.3MB/s 
     |████████████████████████████████| 3.4MB 66.1MB/s 
     |████████████████████████████████| 235kB 56.3MB/s 
     |████████████████████████████████| 747kB 57.1MB/s 
     |████████████████████████████████| 25.3MB 1.3MB/s 
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
     |████████████████████████████████| 1.7MB 7.7MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 163kB 52.8MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# drive.flush_and_unmount()
!pwd

Mounted at /content/drive
/content


In [ ]:
# For Vit importing
import timm

# For ViT examples
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# General + imagenet dataloader
from __future__ import print_function, division
import torchvision
from torchvision import datasets, models, transforms
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time 
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

plt.ion()   # interactive mode

# ART lib
from art.attacks.evasion import FastGradientMethod, DeepFool
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

# Progress bar
# Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm

# foolbox
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import L2CarliniWagnerAttack, LinfPGD, LinfFastGradientAttack, L2PGD
import foolbox as fb


In [ ]:
#@title helper functions
def convert_1hot(data, num_classes=100):
  rows = len(data)
  cols = num_classes
  new_data = np.zeros((rows,cols))
  for idx, i in enumerate(data):
    new_data[idx,i] = 1
  return new_data


def count_parameters(model):
  num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print("The model has {} parameters".format(num_params))
  return num_params

def clear_pytorch_cache():
  torch.cuda.empty_cache()


def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


def show_cached_memory():
  print("Memory cached : {} MB".format(torch.cuda.memory_cached() / 1024**2))


# save image to folders
def save_adv_images(inputs, clipped_advs, labels, model, dirpath=""):

  dir = dirpath
  if len(dirpath) == "":
    timestr = time.strftime("%Y%m%d-%H%M%S")
    dirpath = os.path.join("/content/drive/MyDrive/598dataset", timestr)

  
  # try to create the dir
  os.makedirs(dir, exist_ok=True)

  # loop over images and save adv results
  clean_preds = model(inputs).argmax(axis=1)
  adv_preds = model(clipped_advs).argmax(axis=1)
  norm = ((inputs - clipped_advs)**2).sum(axis=(1, 2, 3))**0.5
  norm = norm.cpu().numpy().round(2)

  # remember dir count
  filenum_counter = {}

  for i in range(inputs.shape[0]):

    adv_img = clipped_advs[i].cpu().numpy()
    adv_img = adv_img.swapaxes(0, 1)
    adv_img = adv_img.swapaxes(1, 2)

    # get labels and predictions
    label = class_names[int(labels[i])]
    clean_pred = class_names[int(clean_preds[i])]
    adv_pred = class_names[int(adv_preds[i])]
    img_dir = os.path.join(dir, str(label))

    if img_dir not in filenum_counter:
      os.makedirs(img_dir, exist_ok=True)
      filenum_counter[img_dir] = len(os.listdir(img_dir)) + 1
    else:
      filenum_counter[img_dir] += 1

    # image name
    img_name = "img{}_clean{}_adv{}_L2diff{}.png".format(filenum_counter[img_dir], clean_pred, adv_pred, norm[i])
    img_path = os.path.join(img_dir, img_name)

    # save the image
    # print(adv_img.max(), adv_img.min(), adv_img.shape)
    plt.imsave(img_path, adv_img)

    # load the image and test

    

def torch_batch_to_img_array(batch):

  # render into numpy
  if str(type(batch)) == "<class 'list'>":
    batch = batch[0]
  
  # if 
  # batch = batch.cpu().numpy()


## 2.

In [ ]:
# Config
NUM_CLASSES = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# clear the cache of cuda
clear_pytorch_cache()

# Load imagenet tiny, transformer in validation set enabled
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/598dataset/tinyImageNet/TinyImageNet'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, "tiny_"+x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# update num classes based on real number of classes
NUM_CLASSES = len(class_names)


In [ ]:
class EnsembleNet(nn.Module):
  def __init__(self):
    super(EnsembleNet, self).__init__()

    self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
    self.vit.head = nn.Linear(self.vit.head.in_features, NUM_CLASSES) 
    self.vit.load_state_dict(torch.load('/content/drive/MyDrive/598dataset/ViTbase_16_224_tiny_15classes.pth'))

    self.resnet = models.resnet50(pretrained=True, progress=True)
    self.resnet.fc = nn.Linear(self.resnet.fc.in_features, NUM_CLASSES)
    self.resnet.load_state_dict(torch.load('/content/drive/MyDrive/598dataset/ResNet50_tinyImgNet.pth'))

  def forward(self, x):
    x1 = self.vit(x)
    x2 = self.resnet(x)
    return (x1 + x2)/2
    # return x1

In [ ]:
model = EnsembleNet()
model.to(device)
model.eval()


preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

count = 0
running_acc = 0


show_cached_memory()
with tqdm(dataloaders['val'], unit='batch') as tepoch:
  for inputs, labels in tepoch:
    inputs = inputs.to(device)
    labels = labels.to(device)
    running_acc += accuracy(fmodel, inputs, labels)
    count += 1

print(f"clean accuracy:  {running_acc/count * 100:.1f} %")

Memory cached : 460.0 MB


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)



clean accuracy:  93.1 %


In [ ]:
# load the data
# clear the cache
clear_pytorch_cache()

# Load imagenet tiny, transformer in validation/train set disabled
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/598dataset/tinyImageNet/TinyImageNet'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, "tiny_"+x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get some constants
NUM_CLASSES = len(class_names)

def test_attack(attack, epsilons, fmodel, dataloaders, img_save_dir="", save_train=False):
  count = 0
  running_acc = torch.zeros(len(epsilons)).to(device)

  # Vlidation set
  with tqdm(dataloaders['val'], unit='batch') as tepoch:
    for inputs, labels in tepoch:
      inputs = inputs.to(device)
      labels = labels.to(device)
      raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)

      # save images
      if img_save_dir:
        for idx in range(len(clipped_advs)):
          dirname = "Epsilon{}".format(epsilons[idx]) + "/val"
          save_adv_images(inputs, clipped_advs[idx], labels, fmodel, dirpath=os.path.join(img_save_dir, dirname))

      running_acc += 1 - success.float().mean(axis=-1)
      count += 1
      clear_pytorch_cache()
      show_cached_memory()

  # report accuracy on val dataset
  running_acc /= count
  for eps, acc in zip(epsilons, running_acc):
      print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")
  
  # save train adv
  if save_train:
    with tqdm(dataloaders['train'], unit='batch') as tepoch:
      for inputs, labels in tepoch:
        inputs = inputs.to(device)
        labels = labels.to(device)
        raw_advs, clipped_advs, success = attack(fmodel, inputs, labels, epsilons=epsilons)

        # save images
        if img_save_dir:
          for idx in range(len(clipped_advs)):
            dirname = "Epsilon{}".format(epsilons[idx]) + "/train"
            save_adv_images(inputs, clipped_advs[idx], labels, fmodel, dirpath=os.path.join(img_save_dir, dirname))



In [ ]:
# FGSM, Linf
attack = LinfFastGradientAttack()
epsilons = [
    0.001,
    0.005,
    0.01,
    0.1
]
img_save_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/FGSM_formal'

# Note that for the training dir to be saved, normalization should be disabled
test_attack(attack, epsilons, fmodel, dataloaders, img_save_dir, save_train=False)


  Linf norm ≤ 0.001 : 75.7 %
  Linf norm ≤ 0.005 : 46.7 %
  Linf norm ≤ 0.01  : 38.7 %
  Linf norm ≤ 0.1   : 30.7 %


In [ ]:
# L2 PGD, Epsilons need to be larger
attack = L2PGD()
epsilons = [
    0.1,
    0.3,
    0.5,
    1,
    2,
    5
]

img_save_dir = '/content/drive/MyDrive/598dataset/TinyImgNet_AdvSamples/Ensemble/PGD_L2_Formal'
test_attack(attack, epsilons, fmodel, dataloaders, img_save_dir, save_train=False)

/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


Memory cached : 3614.0 MB
Memory cached : 3614.0 MB
Memory cached : 2720.0 MB
Memory cached : 1852.0 MB
Memory cached : 4666.0 MB
Memory cached : 3220.0 MB
Memory cached : 4314.0 MB
Memory cached : 3274.0 MB
Memory cached : 2290.0 MB
Memory cached : 4824.0 MB
Memory cached : 3590.0 MB
Memory cached : 2530.0 MB
Memory cached : 1622.0 MB
Memory cached : 4288.0 MB
Memory cached : 3238.0 MB
Memory cached : 2016.0 MB
Memory cached : 4624.0 MB
Memory cached : 3190.0 MB
Memory cached : 1856.0 MB
Memory cached : 4302.0 MB
Memory cached : 2936.0 MB
Memory cached : 1680.0 MB
Memory cached : 4728.0 MB
Memory cached : 3270.0 MB
Memory cached : 2030.0 MB
Memory cached : 4566.0 MB
Memory cached : 3316.0 MB
Memory cached : 2484.0 MB
Memory cached : 1676.0 MB
Memory cached : 4674.0 MB
Memory cached : 3230.0 MB
Memory cached : 2062.0 MB
Memory cached : 4846.0 MB
Memory cached : 3390.0 MB
Memory cached : 2336.0 MB
Memory cached : 4986.0 MB
Memory cached : 3606.0 MB
Memory cached : 2684.0 MB
Memory cache